In [7]:
--Cleaning data on alternative fuel stations using queries in SQL Server 
--Data from https://afdc.energy.gov/data_download

--Initial query to take a look at the data
SELECT * FROM dbo.alt_fuel_stations
ORDER BY ID
OFFSET 0 ROWS FETCH FIRST 10 ROWS ONLY;

(10 rows affected)

Total execution time: 00:00:00.116

Fuel_Type_Code,Station_Name,Street_Address,Intersection_Directions,City,State,ZIP,Plus4,Station_Phone,Status_Code,Expected_Date,Groups_With_Access_Code,Access_Days_Time,Cards_Accepted,BD_Blends,NG_Fill_Type_Code,NG_PSI,EV_Level1_EVSE_Num,EV_Level2_EVSE_Num,EV_DC_Fast_Count,EV_Other_Info,EV_Network,EV_Network_Web,Geocode_Status,Latitude,Longitude,Date_Last_Confirmed,ID,Updated_At,Owner_Type_Code,Federal_Agency_ID,Federal_Agency_Name,Open_Date,Hydrogen_Status_Link,NG_Vehicle_Class,LPG_Primary,E85_Blender_Pump,EV_Connector_Types,Country,Intersection_Directions_French,Access_Days_Time_French,BD_Blends_French,Groups_With_Access_Code_French,Hydrogen_Is_Retail,Access_Code,Access_Detail_Code,Federal_Agency_Code,Facility_Type,CNG_Dispenser_Num,CNG_On_Site_Renewable_Source,CNG_Total_Compression_Capacity,CNG_Storage_Capacity,LNG_On_Site_Renewable_Source,E85_Other_Ethanol_Blends,EV_Pricing,EV_Pricing_French,LPG_Nozzle_Types,Hydrogen_Pressures,Hydrogen_Standards,CNG_Fill_Type_Code,CNG_PSI,CNG_Vehicle_Class,LNG_Vehicle_Class,EV_On_Site_Renewable_Source,Restricted_Access,RD_Blends,RD_Blends_French,RD_Blended_with_Biodiesel,RD_Maximum_Biodiesel_Level,NPS_Unit_Name,CNG_Station_Sells_Renewable_Natural_Gas,LNG_Station_Sells_Renewable_Natural_Gas
CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NULL,Montgomery,AL,36107,NULL,NULL,E,NULL,Private,NULL,NULL,NULL,B,3600,NULL,NULL,NULL,NULL,NULL,NULL,200-9,32.367916,-86.267021,2023-04-06,17,2023-04-06 17:17:01 UTC,T,NULL,NULL,2010-12-01,NULL,MD,NULL,NULL,NULL,US,NULL,NULL,NULL,Privé,NULL,private,NULL,NULL,STANDALONE_STATION,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,B,3600,MD,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,false,NULL
CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NULL,Atlanta,GA,30324,NULL,NULL,E,NULL,Private - Government only,NULL,NULL,NULL,Q,3000,NULL,NULL,NULL,NULL,NULL,NULL,200-8,33.821911,-84.367461,2023-01-10,45,2023-02-14 15:54:11 UTC,LG,NULL,NULL,1996-12-15,NULL,LD,NULL,NULL,NULL,US,NULL,NULL,NULL,Privé - Réservé au gouvernement,NULL,private,GOVERNMENT,NULL,FLEET_GARAGE,NULL,NULL,30,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Q,3000,LD,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
CNG,United Parcel Service,270 Marvin Miller Dr,NULL,Atlanta,GA,30336,NULL,NULL,E,NULL,Private,NULL,NULL,NULL,B,3600,NULL,NULL,NULL,NULL,NULL,NULL,200-9,33.760256,-84.543822,2022-06-14,64,2023-02-14 15:54:11 UTC,P,NULL,NULL,1997-01-01,NULL,HD,NULL,NULL,NULL,US,NULL,NULL,NULL,Privé,NULL,private,NULL,NULL,STANDALONE_STATION,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,B,3600,HD,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
CNG,Arkansas Oklahoma Gas Corp,2100 S Waldron Rd,NULL,Fort Smith,AR,72903,NULL,479-783-3181,E,NULL,Public - Credit card at all times,24 hours daily,FuelMan M V Wright_Exp,NULL,Q,3600,NULL,NULL,NULL,NULL,NULL,NULL,200-9,35.362213,-94.375338,2023-02-13,73,2023-02-14 15:54:11 UTC,T,NULL,NULL,1997-01-01,NULL,MD,NULL,NULL,NULL,US,NULL,NULL,NULL,Public - Carte de crédit en tout temps,NULL,public,CREDIT_CARD_ALWAYS,NULL,UTILITY,1,NULL,250,57855,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Q,3600,MD,NULL,NULL,false,NULL,NULL,NULL,NULL,NULL,false,NULL
CNG,Clean Energy - Logan International Airport,1000 Cottage St Ext,"From Route 1, take the first exit after Callahan Tunnel. Located near the Massachusetts State Police Troop F building on Service Rd.",East Boston,MA,02128,NULL,866-809-4869,E,NULL,Public - Credit card at all times,24 hours daily; call 866-809-4869 for Clean Energy card,A CleanEnergy Comdata D FuelMan M V Voyager Wright_Exp,NULL,Q,3000 3600,NULL,NULL,NULL,NULL,NULL,NULL,GPS,42.37470554648246,-71.02654860574343,2022-12-13,81,2023-02-14 15:54:11 UTC,SG,NULL,NULL,1996-11-15,NULL,MD,NULL,NULL,NULL,US,NULL,NULL,NULL,Public - Carte de crédit en tout temps,NULL,public,CREDIT_CARD_ALWAYS,NULL,AIRPORT,4,NONE,850,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Q,3000 3600,MD,NULL,NULL,false,NULL,NULL,NULL,NULL,NULL,true,NULL
CNG,Clean Energy - Everett - Nationa

In [8]:
--Make a new table containing only the desired columns 
--about station type, location, and access using DISTINCT to not include duplicates
DROP Table if exists dbo.alt_fuel_stations_clean
SELECT DISTINCT ID, Fuel_type_code, Station_Name, City, State, ZIP, Country, 
    Status_Code, Access_Code, Facility_Type 
    INTO dbo.alt_fuel_stations_clean FROM dbo.alt_fuel_stations
    WHERE Status_Code is not NULL and Access_Code is not NULL

(71187 rows affected)

Total execution time: 00:00:00.400

In [11]:
--Checking for null values in State, ZIP, Latitude, and Longitude fields since I am interested in the geographic distribution of fueling stations
SELECT City, State, ZIP, Country from dbo.alt_fuel_stations_clean
    WHERE State is NULL or ZIP is NULL or Country is NULL
    ORDER BY ID

(31 rows affected)

Total execution time: 00:00:00.039

City,State,ZIP,Country
Long Beach,NULL,90808,US
Phoenix,NULL,85015,US
Goodyear,NULL,85395,US
Orlando,NULL,32817,US
Santa Monica,NULL,90403,US
Portland,NULL,97225,US
Merced,NULL,95343,US
Northfield,NULL,60093,US
Austin,NULL,78758,US
Frisco,NULL,75035,US


In [12]:
--Looks there are no null country or zip code values, but there are null states. 
--However, I can fill the missing state values in from the zip codes using a table 
--imported from https://simplemaps.com/data/us-zips

UPDATE dbo.alt_fuel_stations_clean
SET State = COALESCE(State, 
    (SELECT state_name FROM dbo.uszips 
        WHERE dbo.uszips.zip = dbo.alt_fuel_stations_clean.ZIP),'Unknown')


(71187 rows affected)

Total execution time: 00:00:00.148

In [13]:
--Replace null Facility_Type values with "unknown"
UPDATE dbo.alt_fuel_stations_clean
SET Facility_Type = COALESCE(Facility_Type,'unknown')

(71187 rows affected)

Total execution time: 00:00:00.385

In [14]:
--Make new column with the full name of the status based on the code
ALTER TABLE dbo.alt_fuel_stations_clean
ADD Status NVARCHAR(255);

Commands completed successfully.

Total execution time: 00:00:00.024

In [15]:
--Replace status codes with written out descriptions for increased readability
UPDATE dbo.alt_fuel_stations_clean
SET Status = 
    CASE  WHEN Status_Code = 'E' THEN 'Available'
          WHEN Status_Code = 'P' THEN 'Planned'
          ELSE 'Temporarily Unavailable'
          END

(71187 rows affected)

Total execution time: 00:00:00.560

In [16]:
--Make new column with the full name of the fuel type
ALTER TABLE dbo.alt_fuel_stations_clean
ADD Fuel_Type NVARCHAR(255);

Commands completed successfully.

Total execution time: 00:00:00.011

In [17]:
--Fill new column with the fuel type description from the dataset documentation
UPDATE dbo.alt_fuel_stations_clean
SET Fuel_type = CASE WHEN Fuel_type_code = 'BD' THEN 'Biodiesel (B20 and above)'
    WHEN Fuel_type_code = 'CNG' THEN 'Compressed Natural Gas'
    WHEN Fuel_type_code = 'ELEC' THEN 'Electric'
    WHEN Fuel_type_code = 'E85' THEN 'Ethanol (E85)'
    WHEN Fuel_type_code = 'HY' THEN 'Hydrogen'
    WHEN Fuel_type_code = 'LNG' THEN 'Liquefied Natural Gas'
    WHEN Fuel_type_code = 'LPG' THEN 'Propane (LPG)'
    WHEN Fuel_type_code = 'RD' THEN 'Renewable Diesel (R20 and above)'
    END

(71187 rows affected)

Total execution time: 00:00:00.570

In [18]:
--Trimming trailing spaces and removing underscores from Facility type, and making lowercase
UPDATE dbo.alt_fuel_stations_clean
SET Facility_Type = REPLACE(LOWER(TRIM(Facility_Type)), '_', ' ')

(71187 rows affected)

Total execution time: 00:00:00.195

In [19]:
--Check to make sure the data look good
SELECT * FROM dbo.alt_fuel_stations_clean
ORDER BY ID
OFFSET 0 ROWS FETCH FIRST 10 ROWS ONLY;

(10 rows affected)

Total execution time: 00:00:00.042

ID,Fuel_type_code,Station_Name,City,State,ZIP,Country,Status_Code,Access_Code,Facility_Type,Status,Fuel_Type
17,CNG,Spire - Montgomery Operations Center,Montgomery,AL,36107,US,E,private,standalone station,Available,Compressed Natural Gas
45,CNG,Metropolitan Atlanta Rapid Transit Authority,Atlanta,GA,30324,US,E,private,fleet garage,Available,Compressed Natural Gas
64,CNG,United Parcel Service,Atlanta,GA,30336,US,E,private,standalone station,Available,Compressed Natural Gas
73,CNG,Arkansas Oklahoma Gas Corp,Fort Smith,AR,72903,US,E,public,utility,Available,Compressed Natural Gas
81,CNG,Clean Energy - Logan International Airport,East Boston,MA,02128,US,E,public,airport,Available,Compressed Natural Gas
84,CNG,Clean Energy - Everett - National Grid,Everett,MA,02149,US,E,public,standalone station,Available,Compressed Natural Gas
108,CNG,Clean Energy - Greenpoint - National Grid,Brooklyn,NY,11211,US,E,public,utility,Available,Compressed Natural Gas
112,CNG,Canarsie - National Grid,Brooklyn,NY,11236,US,E,public,utility,Available,Compressed Natural Gas
124,CNG,Con Edison - W 29th St Service Center,New York,NY,10001,US,E,public,utility,Available,Compressed Natural Gas
125,CNG,Con Edison - E 16th St Service Center,New York,NY,10009,US,E,public,utility,Available,Compressed Natural Gas
